# 1. Import and Install Dependencies

In [ ]:
! pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# 3. Extract Keypoint Values

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

# 4. Setup Folders for Collection

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['','I','You','Love','Hello','Namaste', 'Bye', 
                    'Thanks', 'Welcome', 'Indian', 'Good Morning','Good Afternoon', 
                    'Good night','Sorry','Please','Car','Food','Water','Today',
                    'Tomorrow','Time','Family','Mother','Father','Tree','House','Beautiful',
                    'Yes','No','Deaf'])
# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length 
sequence_length = 30

In [8]:
# Deleting folder of data
import shutil
 
dir = os.path.join(DATA_PATH)
shutil.rmtree(dir)

In [9]:
for action in actions: 
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        ret, frame = cap.read()
        cv2.putText(frame, f'{action}', (120,200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 3, (255,0,0), 5, cv2.LINE_AA)
        cv2.imshow('OpenCV Feed', frame)
        cv2.waitKey(2000)
        # Loop through sequences aka videos
        for sequence in range(1,sequence_length+1):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                         
    cap.release()
    cv2.destroyAllWindows()

In [143]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'': 0,
 'I': 1,
 'You': 2,
 'Love': 3,
 'Hello': 4,
 'Namaste': 5,
 'Bye': 6,
 'Thanks': 7,
 'Welcome': 8,
 'Indian': 9,
 'Good Morning': 10,
 'Good Afternoon': 11,
 'Good night': 12,
 'Sorry': 13,
 'Please': 14,
 'Car': 15,
 'Food': 16,
 'Water': 17,
 'Today': 18,
 'Tomorrow': 19,
 'Time': 20,
 'Family': 21,
 'Mother': 22,
 'Father': 23,
 'Tree': 24,
 'House': 25,
 'Beautiful': 26,
 'Yes': 27,
 'No': 28,
 'Deaf': 29}

In [16]:
sequences, labels = [], []
for action in actions:
    for sequence in range(1,no_sequences+1):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [17]:
X = np.array(sequences)

In [18]:
X.shape

(900, 30, 258)

In [19]:
y = to_categorical(labels).astype(int)

In [24]:
#y

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [23]:
X_train.shape

(810, 30, 258)

# 7. Build and Train LSTM Neural Network

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

In [14]:
log_dir = os.path.join('Logs1')
tb_callback = TensorBoard(log_dir=log_dir)

NameError: name 'TensorBoard' is not defined

In [77]:
from tensorflow.keras.layers import Dropout

In [12]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(30, activation='softmax'))

In [150]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_36 (LSTM)              (None, 30, 64)            82688     
                                                                 
 lstm_37 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_38 (LSTM)              (None, 64)                49408     
                                                                 
 dense_34 (Dense)            (None, 64)                4160      
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_35 (Dense)            (None, 32)                2080      
                                                                 
 dropout_14 (Dropout)        (None, 32)              

In [16]:
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [121]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

In [116]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [122]:
history = model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test), verbose=1)

Epoch 1/150


26/26 [==============================] - 8s 204ms/step - loss: 1.3805 - categorical_accuracy: 0.5185 - val_loss: 1.1095 - val_categorical_accuracy: 0.6444
Epoch 2/150
26/26 [==============================] - 5s 181ms/step - loss: 0.8732 - categorical_accuracy: 0.6988 - val_loss: 0.8278 - val_categorical_accuracy: 0.6889
Epoch 3/150
26/26 [==============================] - 5s 196ms/step - loss: 0.7835 - categorical_accuracy: 0.7160 - val_loss: 0.8053 - val_categorical_accuracy: 0.6556
Epoch 4/150
26/26 [==============================] - 6s 215ms/step - loss: 0.8014 - categorical_accuracy: 0.6975 - val_loss: 1.0206 - val_categorical_accuracy: 0.6778
Epoch 5/150
26/26 [==============================] - 7s 260ms/step - loss: 0.6846 - categorical_accuracy: 0.7556 - val_loss: 0.6480 - val_categorical_accuracy: 0.7889
Epoch 6/150
26/26 [==============================] - 6s 235ms/step - loss: 0.8078 - categorical_accuracy: 0.7136 - val_loss: 0.9872 - val_categorical_accuracy: 0.7333
Epoch 7/15

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['categorical_accuracy'], label='Training Accuracy')
plt.plot(history.history['val_categorical_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
%tensorboard --logdir Logs1

# 8. Make Predictions

In [39]:
res = model.predict(X_test)

3/3 [==============================] - 1s 13ms/step


In [40]:
for i in range(5):
    print(actions[np.argmax(res[i])])

Good night
Thanks
Food
I
Good night


In [41]:
for i in range(5):
    print(actions[np.argmax(y_test[i])])

Good night
Thanks
Food
I
Good night


# 9. Save Weights

In [110]:
model.save('30_actions.h5')

In [13]:
model.load_weights('30_actions.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [43]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [44]:
yhat = model.predict(X_train)

26/26 [==============================] - 1s 16ms/step


In [45]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [46]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[785,   0],
        [  0,  25]],

       [[785,   0],
        [  0,  25]],

       [[785,   0],
        [  0,  25]],

       [[784,   0],
        [  0,  26]],

       [[781,   0],
        [  0,  29]],

       [[782,   0],
        [  0,  28]],

       [[783,   0],
        [  0,  27]],

       [[784,   0],
        [  0,  26]],

       [[784,   0],
        [  0,  26]],

       [[782,   0],
        [  0,  28]],

       [[784,   0],
        [  0,  26]],

       [[781,   0],
        [  0,  29]],

       [[782,   0],
        [  0,  28]],

       [[784,   0],
        [  0,  26]],

       [[783,   0],
        [  0,  27]],

       [[784,   0],
        [  0,  26]],

       [[783,   0],
        [  0,  27]],

       [[782,   0],
        [  0,  28]],

       [[783,   0],
        [  0,  27]],

       [[782,   0],
        [  0,  28]],

       [[785,   0],
        [  0,  25]],

       [[782,   0],
        [  0,  28]],

       [[782,   0],
        [  0,  28]],

       [[782,   0],
        [  0, 

In [47]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [44]:
# from scipy import stats

In [14]:
colors = []
for _ in range(len(label_map)):
    num = np.random.randint(0,256)
    colors.append((num,num,num))

In [15]:
# colors

In [16]:
colors = []
for _ in range(len(label_map)):
    num = np.random.randint(0,256)
    colors.append((num,num,255))

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [30]:
# plt.figure(figsize=(18,18))
# plt.imshow(prob_viz(res, actions, image, colors))

In [46]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.3

timer = 30

# Function to display the timer on the frame
def display_timer(frame, timer):
    cv2.putText(frame, "{}".format(timer), (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        display_timer(frame, timer)
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            sequence = []
            
        # 3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 3: 
                sentence = sentence[-3:]

            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        timer -= 1

        if timer == 0:
            cv2.waitKey(1000)
            timer = 30
            
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 22ms/step
